# Example Using Sycamore Quantum Circuit

Here we'll run through a more in-depth tensor contraction path finding, including all the different visualization options, by computing some amplitudes for random circuits on Google's Sycamore chip .  

In [1]:
import quimb.tensor as qtn
import cotengra as ctg

In [2]:
# just set up some misc notebook plotting stuff

%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

Two Sycamore circuit definitions are included in this repository, the first of which ($m=10$) should fit into memory, and the second of which ($m=12$) will require *slicing*.

In [3]:
def load_circuit(
    n=53,
    depth=10,
    seed=0 ,
    elided=0,
    sequence='ABCDCDAB',
    swap_trick=False
):
    file = f'circuit_n{n}_m{depth}_s{seed}_e{elided}_p{sequence}.qsim'

    if swap_trick:
        gate_opts={'contract': 'swap-split-gate', 'max_bond': 2}  
    else:
        gate_opts={}
    
    # instantiate the `Circuit` object that 
    # constructs the initial tensor network:
    return qtn.Circuit.from_qasm_file(file, gate_opts=gate_opts)

Make our target tensor network the overlap of the wavefunction with a bitstring:

In [4]:
circ = load_circuit(depth=20)
psi_f = qtn.MPS_computational_state('0' * (circ.N))
tn = circ.psi & psi_f
output_inds = []

We can check out what the raw TN looks like:

In [5]:
#tn.graph(iterations=20, color=circ.psi.site_tags, legend=False, figsize=(3, 3))

As well as what it looks like after standard pre-processing:

In [6]:
# inplace full simplify and cast to single precision
tn.full_simplify_(output_inds=output_inds)
tn.astype_('complex64')

<TensorNetwork1D(tensors=381, indices=754, L=53, max_bond=2)>

The simplification uses some `numba` compiled functions which might slow things done first run.

In [7]:
#tn.graph(initial_layout='kamada_kawai', iterations=10, color=circ.psi.site_tags, legend=False, figsize=(3, 3))

Now we're ready to try and find a contraction path (various initializiation options are illustrated - not necessarily the best):

In [8]:
opt = ctg.HyperOptimizer(
    methods=['cyc_kahypar-balanced', 'kahypar-balanced', 'cyc_kahypar', 'kahypar'],
    max_repeats=500,
    progbar=True,
    minimize='size',#'flops
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)

The optimizer is stateful, so this following actual search call can be run repeatedly:

In [9]:
#info = tn.contract(all, optimize=opt, get='path-info')

We can visualize the progress of the Bayesian optimizer like so:

In [10]:
opt = ctg.HyperOptimizer(
    methods=['kahypar'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 53.00 log10[FLOPs]: 18.90: 100%|██| 128/128 [02:48<00:00,  1.32s/it]


In [11]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 53.00 log10[FLOPs]: 24.08: : 500it [00:04, 100.83it/s]
log2[SIZE]: 53.00 log10[FLOPs]: 18.89: : 500it [00:04, 114.71it/s]


In [12]:
opt = ctg.HyperOptimizer(
    methods=['kahypar-balanced'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.70: 100%|██| 128/128 [02:18<00:00,  1.08s/it]


In [13]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.83: : 500it [00:05, 86.72it/s] 
log2[SIZE]: 52.00 log10[FLOPs]: 18.67: : 500it [00:04, 109.10it/s]


In [14]:
opt = ctg.HyperOptimizer(
    methods=['kahypar-agglom'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.68: 100%|██| 128/128 [02:14<00:00,  1.05s/it]


In [15]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.36: : 500it [00:05, 89.68it/s] 
log2[SIZE]: 52.00 log10[FLOPs]: 18.68: : 500it [00:04, 113.34it/s]


In [16]:
opt = ctg.HyperOptimizer(
    methods=['cyc_kahypar'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 20.07: 100%|██| 128/128 [02:39<00:00,  1.25s/it]


In [17]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.29: : 500it [00:04, 105.90it/s]
log2[SIZE]: 52.00 log10[FLOPs]: 20.07: : 500it [00:05, 97.94it/s] 


In [18]:
opt = ctg.HyperOptimizer(
    methods=['cyc_kahypar-balanced'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.82: 100%|██| 128/128 [02:16<00:00,  1.07s/it]


In [19]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')#,subtree_size=12

log2[SIZE]: 52.00 log10[FLOPs]: 23.67: : 500it [00:05, 92.49it/s] 
log2[SIZE]: 52.00 log10[FLOPs]: 18.76: : 500it [00:04, 103.03it/s]


In [20]:
opt = ctg.HyperOptimizer(
    methods=['cyc_kahypar-agglom'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    #slicing_reconf_opts={'target_size': 2**27, 'reconf_opts': {}},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.71: 100%|██| 128/128 [02:14<00:00,  1.05s/it]


In [21]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.90: : 500it [00:05, 95.33it/s] 
log2[SIZE]: 52.00 log10[FLOPs]: 18.70: : 500it [00:04, 109.49it/s]


In [22]:
opt = ctg.HyperOptimizer(
    methods=['cyc2_kahypar'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.61: 100%|██| 128/128 [05:35<00:00,  2.62s/it]


In [23]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.88: : 500it [00:06, 79.75it/s]
log2[SIZE]: 52.00 log10[FLOPs]: 18.59: : 500it [00:06, 77.00it/s]


In [24]:
opt = ctg.HyperOptimizer(
    methods=['cyc2_kahypar-balanced'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.40: 100%|██| 128/128 [05:52<00:00,  2.76s/it]


In [25]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 24.11: : 500it [00:04, 120.93it/s]
log2[SIZE]: 52.00 log10[FLOPs]: 18.39: : 500it [00:04, 117.54it/s]


In [26]:
opt = ctg.HyperOptimizer(
    methods=['cyc2_kahypar-agglom'],
    max_repeats=128,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    #slicing_reconf_opts={'target_size': 2**27, 'reconf_opts': {}},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

log2[SIZE]: 52.00 log10[FLOPs]: 18.70: 100%|██| 128/128 [02:17<00:00,  1.07s/it]


In [27]:
tree = ctg.ContractionTree.from_info(info)
tree_s = tree.subtree_reconfigure(progbar=True,minimize='size')
tree_f = tree.subtree_reconfigure(progbar=True,minimize='flops')

log2[SIZE]: 52.00 log10[FLOPs]: 23.81: : 500it [00:04, 113.95it/s]
log2[SIZE]: 52.00 log10[FLOPs]: 18.69: : 500it [00:04, 109.43it/s]


In [ ]:
opt = ctg.HyperOptimizer(
    methods=['cyc_kahypar', 'cyc_kahypar-balanced', 'cyc_kahypar-agglom', 'cyc2_kahypar', 'cyc2_kahypar-balanced', 'cyc2_kahypar-agglom', 'kahypar', 'kahypar-balanced', 'kahypar-agglom'],
    max_repeats=1500,
    progbar=True,
    #optlib = 'random',
    minimize='size',#'flops
    reconf_opts={'minimize':'flops'},
    score_compression=0.5,  # deliberately make the optimizer try many methods 
)
info = tn.contract(all, optimize=opt, get='path-info')

In [ ]:
opt.plot_trials()

Clearly the `kahypar` optimizer seems to be able to find the lowest cost contractions.

We can also plot the relationship between contraction flops and size (the `minimize='combo'` score (log2[SIZE] + log2[FLOPS]) effectively ranks how close they are to the origin and can be useful to balance the two aims):

In [ ]:
opt.plot_scatter()

Where it becomes apparent, that while correlated, the minimum size contraction found is not necessarily the same as the minimum cost contraction found.

If we want to visualize what the actual best contraction tree looks like we need to extract the `ContractionTree` object from the optimizer:

In [ ]:
tree = opt.get_tree()
tree.plot_ring(node_scale= 1 / 3, edge_scale=2 / 3)

We can try and plot what this might look like on top of the TN graph arranged properly, though its likely messy...

In [ ]:
tree.plot_tent()

We can see that the contraction found is imbalanced, with small tensors being steadily absorbed into one big tensor.

One more plot function allows one to investigate the actual numbers involved:

In [ ]:
tree.plot_contractions()

Here, 'peak-size' is the memory required for both inputs and the output of each contraction.

Note again that 'flops' defined here assumes real data (as per `opt_einsum` convention), the 'cost' or number of scalar operations, $C$, is generally half this, whereas for quantum circuits with complex tensors, the real FLOPs will be 4x.

We can also actually perform the contraction (this is using a GTX 2070):

In [ ]:
%%timeit
tn.contract(all, optimize=opt.path, backend='jax')

In [ ]:
%%timeit
tn.contract(all, optimize=opt.path, backend='torch')

TN construction and simplification is determinstic in `quimb` so at least in this case we can easily evaluate another amplitude with the same contraction tree:

In [ ]:
tn = (circ.psi & qtn.MPS_rand_computational_state(circ.N, seed=42))
tn.full_simplify_().astype_('complex64')

In [ ]:
%%time
tn.contract(all, optimize=opt.path, backend='jax')

In [ ]:
%%time
tn.contract(all, optimize=opt.path, backend='jax')

# Searching for sliced contractions (Sycamore $m=12$)

To illustrate slicing we'll setup a (much harder!) depth 12 circuit. We'll perform a swapped rank-2 decomposition on the gates (for a not insignificant drop in total fidelity):

In [ ]:
circ = load_circuit(depth=12, swap_trick=True)
sampler = qtn.MPS_computational_state('0' * (circ.N))
tn = circ.psi & sampler
tn.full_simplify_(output_inds=[])
tn.astype_('complex64')

Because of the rank-2 swapped gate decomposition the full simplify function has now found hyperedge introducing diagonal reductions (which is why there are more tensors than indices).

Now when we intialize the hyper optimizer we'll tell it slice each contraction before reporting the cost and size. 

In [ ]:
# we're going to help accelerate the optimizer search by restricting its search space, 
# since highly balanced contraction trees generally slice best:
ctg.hyper._HYPER_SEARCH_SPACE['kahypar']['imbalance']['max'] = 0.1

opt = ctg.HyperOptimizer(
    methods=['kahypar'],
    max_time=120,              # just search for 2 minutes
    max_repeats=1000,
    progbar=True,
    minimize='flops',
    slicing_opts={'target_size': 2**28}
)

In [ ]:
# because of the hyperedges we need to specify no output indices
info = tn.contract(all, optimize=opt, get='path-info', output_inds=[])

Sliced contractions can be more difficult to find, if performance is critical its worth running this for longer, maybe with a large parallel pool supplied to the `parallel=` kwarg. 

We can see that all the contractions are now 'size 28' however:

In [ ]:
opt.plot_scatter()

We can check what this new contraction tree looks like:

In [ ]:
tree = opt.get_tree()
tree.plot_ring(node_scale=1 / 3, edge_scale=2 / 3)

As enforced, its now somewhat more balanced than the $m=10$ tree.

Now we are ready to search properly for the slicing indices, $2^{28}$ should be small enough to fit into no more than 8GB of memory.

In [ ]:
sf = ctg.SliceFinder(info, target_size=2**28)

We can do quite thorough search with different levels of exploration:

In [ ]:
ix_sl, cost_sl = sf.search(temperature=1.0)
ix_sl, cost_sl = sf.search(temperature=0.1)
ix_sl, cost_sl = sf.search(temperature=0.01)

We can also visualise what effect the slicing has had on the total cost (left - starting point, further to the right equals more sliced):

In [ ]:
sf.plot_slicings(color_scheme='plasma')

Here there seems to have been very little theoretical overhead introduced by the slicing, *for this path*. The real slicing overhead is the increase in FLOPs in comparison to best unsliced path (likely v different).


## Performing the sliced contraction

The order of `quimb` tensors and their data is guaranteed to match that used by the `opt_einsum` syntax:

In [ ]:
arrays = [t.data for t in tn] 
sc = sf.SlicedContractor(arrays)

Or we could translate the opt_einsum symbols back into `quimb` indices to handle the contractions in tensor network form (and use ``.cut_iter``).

In [ ]:
[info.quimb_symbol_map[ix] for ix in ix_sl]

The first time a contraction is run by `jax` with a particular shape its compiled, which can take a few seconds:

In [ ]:
backend = 'jax'

In [ ]:
%%time
c = sc.contract_slice(0, backend=backend)

However, the sliced contraction stores the compiled expression and reuses it for each slice:

In [ ]:
import tqdm

In [ ]:
for i in tqdm.tqdm(range(1, sc.nslices)):
    c = c + sc.contract_slice(i, backend=backend)

In [ ]:
c

Again, the TN manipulations are deterministic so we can re-use everything:

In [ ]:
tn = circ.psi & qtn.MPS_rand_computational_state(circ.N, seed=42)
tn.full_simplify_(output_inds=[]).astype_('complex64')

In [ ]:
# update the SlicedContractor's arrays
sc.arrays = tuple(t.data for t in tn)

# perform the contraction
sum(sc.contract_slice(i, backend=backend) for i in tqdm.tqdm(range(sc.nslices)))

In [ ]:
# update the SlicedContractor's arrays
sc.arrays = tuple(t.data for t in tn)

# perform the contraction
sum(sc.contract_slice(i, backend=backend) for i in tqdm.tqdm(range(sc.nslices)))